In [ ]:
#Name: utkarsh Patel  rollno:166  Batch:C6

#Hotel Management System


import psycopg2
from datetime import datetime


conn = psycopg2.connect(
    dbname="hotel",
    user="postgres",
    password="utkarsh2204",
    host="localhost",
    port="5433"
)
cur = conn.cursor()

# Global variable to store the assigned room_id
room_id = None

# Function to display dashboard
def dashboard():
    print("Welcome to Hotel Management System")
    print("1. Create a new account")
    print("2. Check room availability")
    print("3. Make payment")
    print("4. Exit")

# Function to create new acc of customers and ask some personal details 

def new_acc():
    
    global room_id  
    name = input("Enter your name: ")
    while True:
        dob = input("Enter your date of birth (YYYY-MM-DD): ")
        try:
            datetime.strptime(dob, '%Y-%m-%d')
            break
        except ValueError:
            print("Please enter date in proper format (YYYY-MM-DD)")
    citizenship_number = input("Enter your citizenship number  ")
    address = input("Enter your address: ")
    while True:
        phone_number = input("Enter your phone number (10 digits): ")
        if len(phone_number) != 10:
            print("Phone number should be 10 digits.")
        else:
            break
    check_in = input("Enter check-in date (YYYY-MM-DD): ")
    while True:
        try:
            datetime.strptime(check_in, '%Y-%m-%d')
            break
        except ValueError:
            print("Please enter date in proper format (YYYY-MM-DD)")
    room_type()
    
    if room_id:  # Check if room_id is assigned
        # Insert customer details along with the assigned room ID
        cur.execute("INSERT INTO customer (name, dob, citizenship_number, address, phone_number, check_in, room_id) VALUES (%s, %s, %s, %s, %s, %s, %s)", (name, dob, citizenship_number, address, phone_number, check_in, room_id))
        conn.commit()
        print("Account created successfully!")
    else:
        print("Room assignment failed. Please try again.")

        
# Function to select room type and book the room
def room_type():
    global room_id  
    print("Select room type:")
    print("1. Normal room")
    print("2. Executive room")
    print("3. Personal pool ")
   
    choice = int(input("Enter your choice: "))
    if choice == 1:
        room_type = "normal"
    elif choice == 2:
        room_type = "executive"
    elif choice==3:
        room_type="personal pool"
    else:
        print("Invalid choice")
        return
    print("Select AC or Non-AC:")
    print("1. AC room")
    print("2. Non-AC room")
    ac_choice = int(input("Enter your choice: "))
    if ac_choice == 1:
        ac_type = "AC"
    elif ac_choice == 2:
        ac_type = "Non-AC"
    else:
        print("Invalid choice")
        return
    
    # Check the room availability and book the room 
    cur.execute("SELECT room_id FROM rooms WHERE room_type = %s AND ac_type = %s AND booked = false LIMIT 1", (room_type, ac_type))  # Changed LIMIT to 1 since only one room is needed
    available_room = cur.fetchone()
    if available_room:
        room_id = available_room[0]
        print(f"Room {room_id} booked successfully!")
        # Update the room status to booked
        cur.execute("UPDATE rooms SET booked = true WHERE room_id = %s", (room_id,))
        conn.commit()
    else:
        print("Sorry, no available rooms matching your criteria.")

# Function to check room availability and display which type of room and how much is empty
def check_availability():
   
    cur.execute("SELECT COUNT(*) FROM rooms WHERE booked = false")
    total_rooms = cur.fetchone()[0]
    

    cur.execute("SELECT COUNT(*) FROM rooms WHERE booked = false AND room_type = 'normal' AND ac_type = 'AC'")
    normal_ac_rooms = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM rooms WHERE booked = false AND room_type = 'normal' AND ac_type = 'non-AC'")
    normal_non_ac_rooms = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM rooms WHERE booked = false AND room_type = 'executive' AND ac_type = 'AC'")
    executive_ac_rooms = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM rooms WHERE booked = false AND room_type = 'executive' AND ac_type = 'non-AC'")
    executive_non_ac_rooms = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM rooms WHERE booked = false AND room_type = 'personal_pool'")
    personal_pool_rooms = cur.fetchone()[0]
    
    print("Personal Pool rooms available:", personal_pool_rooms)
    
    print("Total rooms available:", total_rooms)
    print("Normal AC rooms available:", normal_ac_rooms)
    print("Normal Non-AC rooms available:", normal_non_ac_rooms)
    print("Executive AC rooms available:", executive_ac_rooms)
    print("Executive AC rooms available:", executive_non_ac_rooms)


# Function to make payment

def payment():
    room_id = input("Enter room ID: ")
    
    cur.execute("SELECT check_in FROM customer WHERE room_id = %s", (room_id,))
    check_in_date = cur.fetchone()[0]
    
    # Calculate number of days stayed
    current_date = datetime.today().date()
    days_stayed = (current_date - check_in_date).days
    
    cur.execute("SELECT price FROM rooms WHERE room_id = %s", (room_id,))
    price = cur.fetchone()[0]
    total_amount = price * days_stayed
    print("Total amount for", days_stayed, "days:", total_amount)
    
    payment_option = input("Select payment option (online/at checkout): ")
    if payment_option.lower() == "online":
       
        print("Online payment completed successfully!")

        cur.execute("UPDATE rooms SET booked = false WHERE room_id = %s", (room_id,))
        conn.commit()
        print("Room is now vacant.")
    elif payment_option.lower() == "at checkout":
        print("Please pay at the checkout counter")

        cur.execute("UPDATE rooms SET booked = false WHERE room_id = %s", (room_id,))
        conn.commit()
        print("Room is now vacant.")
    else:
        print("Invalid payment option")



        
def main():
    while True:
        dashboard()
        choice = int(input("Enter your choice: "))
        if choice == 1:
            new_acc()
        elif choice == 2:
            check_availability()
        elif choice == 3:
            payment()
            
        elif choice == 4:
            print("Thank you for using Hotel Management System!")
            break
        else:
            print("Invalid choice")

main()


Welcome to Hotel Management System
1. Create a new account
2. Check room availability
3. Make payment
4. Exit
Enter your choice: 1
Enter your name: utk
Enter your date of birth (YYYY-MM-DD): 2004-04-22
Enter your citizenship number  wefgfd234t3234
Enter your address: dfgdf wesfdc
Enter your phone number (10 digits): 1234567890
Enter check-in date (YYYY-MM-DD): 2024-01-22
Select room type:
1. Normal room
2. Executive room
3. Personal pool 
Enter your choice: 3
Select AC or Non-AC:
1. AC room
2. Non-AC room
Enter your choice: 1
Room 301 booked successfully!
Account created successfully!
Welcome to Hotel Management System
1. Create a new account
2. Check room availability
3. Make payment
4. Exit
Enter your choice: 1
Enter your name: uttt
Enter your date of birth (YYYY-MM-DD): 2004-04-01
Enter your citizenship number  oiuy34567654
Enter your address: yjgbvjhn hgbv
Enter your phone number (10 digits): 2345678901
Enter check-in date (YYYY-MM-DD): 2024-01-22
Select room type:
1. Normal room
2